In [ ]:
!python --version
!wget http://geneontology.org/gene-associations/gene_association.sgd.gz -O ./data/gene_association.sgd.gz

# From YeastNet to CLIXO term documents

## What's this?

This notebook is for building Elasticsearch index for CLIXO ontology.

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

# This directory should contains all of the YeastNet interaction files
data_path = './data/raw-interactions'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]
files

['all.txt',
 'INT.CC.YeastNet.v3.4345gene.82319link.txt',
 'INT.CX.YeastNet.v3.5730gene.242504link.txt',
 'INT.DC.YeastNet.v3.3679gene.29880link.txt',
 'INT.GN.YeastNet.v3.1863gene.29475link.txt',
 'INT.GT.YeastNet.v3.4365gene.149498link.txt',
 'INT.HT.YeastNet.v3.5487gene.141347link.txt',
 'INT.LC.YeastNet.v3.5293gene.54421link.txt',
 'INT.PG.YeastNet.v3.2463gene.54496link.txt',
 'INT.TS.YeastNet.v3.1101gene.3510link.txt',
 'preds_yeastnet_no_gi_0.04_0.5.txt.propagate.mapping']

## Create single interaction DataFrame

From all interaction fiules, make a table with all interactions

In [5]:
columns = ['source', 'target', 'score']
all_interactions = pd.DataFrame(columns=columns)

for f in files:
    if not f.startswith('INT'):
        continue
    
    int_type = f.split('.')[1]
    df = pd.read_csv(data_path+'/'+ f, delimiter='\t', names=columns)
    df['interaction'] = int_type
    all_interactions = pd.concat([all_interactions, df])

print(all_interactions.shape)
all_interactions.head(10)

(787450, 4)


,interaction,score,source,target
0,CC,4.0971,YBL075C,YBR169C
1,CC,4.0971,YBR019C,YBR020W
2,CC,4.0971,YBR160W,YDL102W
3,CC,4.0971,YBR169C,YPL240C
4,CC,4.0971,YCR096C,YCR097W
5,CC,4.0971,YDL102W,YLR310C
6,CC,4.0971,YDR127W,YNR016C
7,CC,4.0971,YER095W,YLR032W
8,CC,4.0971,YFL009W,YLR229C
9,CC,4.0971,YJR048W,YOR362C


In [6]:
all_interactions.to_csv('./data/raw-interactions/all.txt', sep='\t')

## Create list of all genes associated with CLIXO terms

In [7]:
# CLIXO term to gene association
mapping = pd.read_csv('./data/raw-interactions/preds_yeastnet_no_gi_0.04_0.5.txt.propagate.mapping', delimiter='\t', names=['gene', 'term'])
mapping.head()

,gene,term
0,YGR220C,10000
1,YPR100W,10000
2,YNR022C,10000
3,YNL252C,10000
4,YCR046C,10000


In [26]:
# All ORF names in CLIXO
mixed_ids = mapping['gene'].unique()
print(mixed_ids.shape)

(5886,)


### Standardize the gene IDs to SGD

In [9]:
# Import gene association file
yeastAnnotationUrl = './data/gene_association.sgd.gz'
cols = pd.read_csv('./annotation_columns.txt', names=['col_names'])
col_names = cols['col_names'].tolist()
yeastAnnotation = pd.read_csv(yeastAnnotationUrl, delimiter='\t', comment='!', compression='gzip', names=col_names)
yeastAnnotation.tail()

,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB:Reference,Evidence,With_or_From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,taxon,Date,Assigned_by,Annotation_Extension,Gene_Product_Form_ID
111269,SGD,S000006732,tX(XXX)L,NaN,GO:0030533,SGD_REF:S000181097|PMID:9023104,ISM,NaN,F,"tRNA of undetermined specificity, predicted by...",tX(XXX)L|tS(GCU)L,gene,taxon:559292,20030507,SGD,NaN,NaN
111270,SGD,S000006732,tX(XXX)L,NaN,GO:0005829,SGD_REF:S000181097|PMID:9023104,IC,GO:0030533,C,"tRNA of undetermined specificity, predicted by...",tX(XXX)L|tS(GCU)L,gene,taxon:559292,20030507,SGD,NaN,NaN
111271,SGD,S000007338,tY(GUA)Q,NaN,GO:0070125,SGD_REF:S000181097|PMID:9023104,IC,GO:0030533,P,Mitochondrial tyrosine tRNA (tRNA-Tyr),tY(GUA)Q,gene,taxon:559292,20150730,SGD,NaN,NaN
111272,SGD,S000007338,tY(GUA)Q,NaN,GO:0005739,SGD_REF:S000181097|PMID:9023104,IC,GO:0030533,C,Mitochondrial tyrosine tRNA (tRNA-Tyr),tY(GUA)Q,gene,taxon:559292,20030507,SGD,NaN,NaN
111273,SGD,S000007338,tY(GUA)Q,NaN,GO:0030533,SGD_REF:S000181097|PMID:9023104,ISM,NaN,F,Mitochondrial tyrosine tRNA (tRNA-Tyr),tY(GUA)Q,gene,taxon:559292,20060721,SGD,NaN,NaN


In [10]:
# Mapping object: from any type of ID to SGD ID
to_sgd = {}

# Annotation for genes
sgd2fullname = {}
sgd2symbol = {}

for row in yeastAnnotation.itertuples():
    sgd = row[2]
    orf = row[3]
    full_name = str(row[10]).replace('\r\n', '')
    syn = str(row[11])
    syns = syn.split('|')
    to_sgd[orf] = sgd
    for synonym in syns:
        to_sgd[synonym] = sgd
    sgd2fullname[sgd] = full_name
    sgd2symbol[sgd] = orf

In [11]:
normalized_map = []

for row in mapping.itertuples():
    gene = row[1]
    term = str(row[2])
    
    # Convert to SGD
    sgd = gene
    if gene in to_sgd.keys():
        sgd = to_sgd[gene]
    entry = (sgd, term)
    normalized_map.append(entry)

In [34]:
# All ORF to SGD
all_sgd = list(map(lambda x: to_sgd[x] if x in to_sgd.keys() else x, mixed_ids))
if len(all_sgd) == len(mixed_ids):
    print('All maped!')
    
# This contains all gene IDs (SGD ID)
uniq_genes = set(all_sgd)
len(uniq_genes)

All maped!


5872

In [36]:
df_genes = pd.DataFrame(list(uniq_genes))

# Save as text file and use it in UNIPROT ID Mapper
df_genes.to_csv('./data/all_sgd.txt', sep='\t', index=False, header=False)

In [40]:
uniprot = pd.read_csv('./data/uniprot-idmapping.txt', delimiter='\t')
print(uniprot.shape)
uniprot.head()

(5874, 16)


,Entry,yourlist:M20170106A7434721E10EE6586998A056CCD0537E5F3872K,Entry name,Gene names (ORF ),Status,Protein names,Gene names,Organism,Length,Pathway,Function [CC],Gene names (synonym ),Gene names (ordered locus ),Gene names (primary ),Keywords,Cross-reference (SGD)
0,P34164,S000003176,SIP2_YEAST,G1155,reviewed,SNF1 protein kinase subunit beta-2 (Protein SP...,SIP2 SPM2 YGL208W G1155,Saccharomyces cerevisiae (strain ATCC 204508 /...,415,NaN,FUNCTION: Beta subunit of the SNF1 kinase comp...,SPM2,YGL208W,SIP2,3D-structure; Cell membrane; Complete proteome...,S000003176;
1,P40356,S000002993,MED3_YEAST,NaN,reviewed,Mediator of RNA polymerase II transcription su...,PGD1 HRS1 MED3 YGL025C,Saccharomyces cerevisiae (strain ATCC 204508 /...,397,NaN,"FUNCTION: Component of the Mediator complex, a...",HRS1 MED3,YGL025C,PGD1,Acetylation; Activator; Complete proteome; Nuc...,S000002993;
2,Q02753,S000000395,RL21A_YEAST,YBR1401,reviewed,60S ribosomal protein L21-A,RPL21A URP1 YBR191W YBR1401,Saccharomyces cerevisiae (strain ATCC 204508 /...,160,NaN,NaN,URP1,YBR191W,RPL21A,3D-structure; Complete proteome; Cytoplasm; Is...,S000000395;
3,P18494,S000000842,GLN3_YEAST,NaN,reviewed,Nitrogen regulatory protein GLN3,GLN3 YER040W,Saccharomyces cerevisiae (strain ATCC 204508 /...,730,NaN,FUNCTION: Positive nitrogen regulatory protein...,NaN,YER040W,GLN3,Activator; Complete proteome; DNA-binding; Met...,S000000842;
4,P53550,S000005062,DCP2_YEAST,N1917,reviewed,m7GpppN-mRNA hydrolase (EC 3.6.1.62) (Protein ...,DCP2 PSU1 YNL118C N1917,Saccharomyces cerevisiae (strain ATCC 204508 /...,970,NaN,FUNCTION: Catalytic component of the decapping...,PSU1,YNL118C,DCP2,3D-structure; Complete proteome; Cytoplasm; Hy...,S000005062;


In [52]:
sgd2orf = {}
for row in uniprot.itertuples():
    sgd = row[2]
    orf = row[13]
    sgd2orf[sgd] = orf

# Test
missing = set()
for sgd in uniq_genes:
    if sgd not in sgd2orf.keys():
        missing.add(sgd)

print(len(missing))
print(missing)

10
{'S000006284', 'S000001767', 'S000002793', 'S000000214', 'S000004975', 'S000000213', 'S000004976', 'S000005659', 'S000003674', 'S000000322'}


In [48]:
idmap = pd.read_csv('./yeast_clean4.txt', delimiter='\t')
idmap.head()

,symbol,locus_name,acc_number,swiss-prot,sgd,sequence_length,3d,chromosome
0,AAC1,YMR056C,P04710,ADT1_YEAST,S000004660,309,13,NaN
1,AAC3,YBR085W,P18238,ADT3_YEAST,S000000289,307,(3),2
2,AAD10,YJR155W,P47182,AAD10_YEAST,S000003916,288,10,NaN
3,AAD14,YNL331C,P42884,AAD14_YEAST,S000005275,376,14,NaN
4,AAD15,YOL165C,Q08361,AAD15_YEAST,S000005525,143,15,NaN


In [54]:
sgd2orf2 = {}
for row in idmap.itertuples():
    sgd = row[5]
    orf = row[2]
    sgd2orf2[sgd] = orf

for sgd in missing:
    sgd2orf[sgd] = sgd2orf2[sgd]

# Test
missing = set()
for sgd in uniq_genes:
    if sgd not in sgd2orf.keys():
        missing.add(sgd)

print(len(missing))
print(len(sgd2orf))

0
5877


## Create mapping from gene to interactions

In [56]:
gene_map = {}
missing_count = 0

all_orf = set(sgd2orf.values())
len(all_orf)

5877

In [57]:
for row in all_interactions.itertuples():
    source = row[3]
    target = row[4]
    
    data = {
        'source': source,
        'target': target,
        'interaction': row[1],
        'score': row[2]
    }    
    
    if source not in all_orf or target not in all_orf:
        missing_count += 1
        continue
        
    interactions = []
    if source in gene_map.keys():
        interactions = gene_map[source]

    interactions.append(data)
    gene_map[source] = interactions

    interactions2 = []
    if target in gene_map.keys():
        interactions2 = gene_map[target]
    
    interactions2.append(data)
    gene_map[target] = interactions2

In [58]:
len(gene_map)

5849

In [67]:
clixo_genes = {}

missing_name = set()

for row in normalized_map:
    sgd = row[0]
    term = str(row[1])
        
    if sgd not in sgd2fullname.keys():
        missing_name.add(sgd2orf[sgd])
        continue
        
    entry = {
        'sgdid': sgd,
        'orf': sgd2orf[sgd],
        'name': sgd2fullname[sgd],
        'symbol': sgd2symbol[sgd]
    }
    
    assigned_genes = []
    if term in clixo_genes.keys():
        assigned_genes = clixo_genes[term]['genes']
    
    assigned_genes.append(entry)
    clixo_genes[term] = {
        'genes': assigned_genes
    }

print(missing_name)

{'YLR466C-B', 'YAR073W', 'YJR114W', 'YLR154W-A', 'YGR045C', 'YDL118W'}


In [74]:
for key in clixo_genes.keys():
        raw_interactions = []
        gene_list = clixo_genes[key]['genes']
        for gene in gene_list:
            sgd = gene['sgdid']
            if sgd in sgd2orf.keys():
                orf = sgd2orf[sgd]
                if orf in gene_map.keys():
                    raw_interactions.append(gene_map[orf])
        
        clixo_genes[key]['interactions'] = raw_interactions[0]

In [76]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# pp.pprint(clixo_genes['10000'])